# LAD names and codes

Local Authority Districts (December 2011) GB BFC (Britain, Full, Clipped)
- https://opendata.arcgis.com/datasets/8c398b60a71d44e2bdf8b5b3bb951988_0.zip?outSR=%7B%22wkid%22%3A27700%2C%22latestWkid%22%3A27700%7D
    
Local Authority Districts (December 2015) Full Clipped Boundaries in Great Britain
- https://opendata.arcgis.com/datasets/8edafbe3276d4b56aec60991cbddda50_0.zip?outSR=%7B%22wkid%22%3A27700%2C%22latestWkid%22%3A27700%7D
    
Local Authority Districts (December 2016) Full Clipped Boundaries in the UK
- https://opendata.arcgis.com/datasets/7ff28788e1e640de8150fb8f35703f6e_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D
    
Local Authority Districts (December 2017) Full Clipped Boundaries in the UK
- https://opendata.arcgis.com/datasets/fab4feab211c4899b602ecfbfbc420a3_0.zip?outSR=%7B%22latestWkid%22%3A4326%2C%22wkid%22%3A4326%7D
    
Local Authority Districts (December 2018) Full Clipped Boundaries UK
- https://opendata.arcgis.com/datasets/b2d5f4f8e9eb469bb22af910bdc1de22_0.zip?outSR=%7B%22wkid%22%3A27700%2C%22latestWkid%22%3A27700%7D

Local Authorities area list (UK, Census Merged, 2011)
- Reference: http://infuse.ukdataservice.ac.uk/help/definitions/2011geographies/index.html
- List: http://infuse.ukdataservice.ac.uk/help/definitions/2011geographies/local-authority-area-list2011.csv
- Boundaries: https://borders.ukdataservice.ac.uk/ukborders/easy_download/prebuilt/shape/infuse_dist_lyr_2011_clipped.zip


Note that Northern Ireland has the largest post-2011 change, with 11 local government districts since 1 April 2015; previously 26 councils since 1973.

In [ ]:
import os

from glob import iglob
from zipfile import ZipFile

import geopandas
import pandas
import requests


In [ ]:
sources = [
    {
        "key": "lad2011",
        "name": "Local Authorities area list for the United Kingdom",
        "url": "https://borders.ukdataservice.ac.uk/ukborders/easy_download/prebuilt/shape/infuse_dist_lyr_2011_clipped.zip",
        "filename": "lad2011.zip"
    },
    {
        "key": "lad2016",
        "name": "Local Authority Districts (December 2016) Ultra Generalised Clipped Boundaries in the UK",
        "url": "https://opendata.arcgis.com/datasets/7ff28788e1e640de8150fb8f35703f6e_4.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D",
        "filename": "lad2016.zip"
    },
    {
        "key": "lad2017",
        "name": "Local Authority Districts (December 2017) Ultra Generalised Clipped Boundaries in the UK",
        "url": "https://opendata.arcgis.com/datasets/fab4feab211c4899b602ecfbfbc420a3_4.zip?outSR=%7B%22latestWkid%22%3A4326%2C%22wkid%22%3A4326%7D",
        "filename": "lad2017.zip"
    },
    {
        "key": "lad2018",
        "name": "Local Authority Districts (December 2018) Ultra Generalised Clipped Boundaries in the UK",
        "url": "https://opendata.arcgis.com/datasets/fef73aeaf13c417dadf2fc99abcf8eef_0.zip?outSR=%7B%22wkid%22%3A27700%2C%22latestWkid%22%3A27700%7D",
        "filename": "lad2018.zip"
    },
    {
        "key": "lut2011",
        "name": "Output Area to Lower Layer Super Output Area to Middle Layer Super Output Area to Local Authority District (December 2011) Lookup in England and Wales",
        "url": "https://opendata.arcgis.com/datasets/6ecda95a83304543bc8feedbd1a58303_0.csv",
        "filename": "oa-lad-lookup.csv"
    }
]

In [ ]:
def download(item):
    if os.path.exists(item['filename']):
        print("Got", item['filename'])
    else:
        print("Downloading", item['filename'])
        r = requests.get(item['url'], stream=True)
        with open(item['filename'], 'wb') as fh:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    fh.write(chunk)

In [ ]:
def extract(item):
    if '.zip' in item['filename']:
        with ZipFile(item['filename']) as zf:
            zf.extractall(item['key'])
        fn = next(iglob(os.path.join(item['key'], '*.shp')))
        df = geopandas.read_file(fn)
    elif '.csv' in item['filename']:
        try:
            df = pandas.read_csv(item['filename'])
        except UnicodeDecodeError:
            df = pandas.read_csv(item['filename'], encoding='ISO-8859-1')
    else:
        df = None
    return df

In [ ]:
data = {}
for item in sources:
    download(item)
    data[item['key']] = extract(item)

In [ ]:
lad11cd_to_census_merged_code = {
    'E06000052': 'E41000052', # Cornwall
    'E06000053': 'E41000052', # Isles of Scilly
    
    'E09000001': 'E41000324', # City of London
    'E09000033': 'E41000324', # Westminster
}

In [ ]:
data['lad2011'].head()

In [ ]:
data['lad2011'].rename(columns={
    'geo_code': 'lad11cd',
    'geo_label': 'lad11nm'
}, inplace=True)

In [ ]:
data['lad2011'].lad11nm = data['lad2011'].lad11nm.str.replace('&', 'and')
data['lad2011'].lad11cd = data['lad2011'].lad11cd.apply(lambda s: s.strip())

In [ ]:
lad11cd = set(data['lad2011'].lad11cd.unique())
lad18cd = set(data['lad2018'].lad18cd.unique())

In [ ]:
subset = (lad11cd - lad18cd).union(lad18cd - lad11cd)
subset

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
nidf = data['lad2011'][data['lad2011'].lad11cd.str.startswith('9')].copy()

In [ ]:
nidf['coords'] = nidf['geometry'].apply(lambda x: x.representative_point().coords[:][0])
nidf.plot(color='#ccccff', edgecolor='white')
for row in nidf.itertuples():
    plt.annotate(s=row.lad11nm, xy=row.coords, horizontalalignment='center')

In [ ]:
ni16 = data['lad2016'][data['lad2016'].lad16cd.str.startswith('N')].copy()

In [ ]:
ni16['coords'] = ni16['geometry'].apply(lambda x: x.representative_point().coords[:][0])
ni16.plot(color='#ccccff', edgecolor='white')
for row in ni16.itertuples():
    plt.annotate(s=row.lad16nm, xy=row.coords, horizontalalignment='center')

In [ ]:
ni116 = geopandas.overlay(ni16, nidf, how='intersection')

In [ ]:
ni116.columns

In [ ]:
ni116['area'] = ni116['geometry'].area

In [ ]:
nimapping = ni116[['lad11nm', 'lad11cd', 'lad16nm', 'lad16cd', 'area']].sort_values(['lad11nm', 'area'])
# keep max area for each lad11
nimapping = nimapping.drop_duplicates(['lad11nm'], keep='last').drop(['area'], axis=1)
nimapping

In [ ]:
len(nimapping), len(nidf), len(ni16), len(nimapping.lad16cd.unique())

In [ ]:
def extract_subset(data, year):
    df = data['lad20{}'.format(year)]
    code = 'lad{}cd'.format(year)
    name = 'lad{}nm'.format(year)
    return df.loc[df[code].isin(subset)][[code, name]]

In [ ]:
changes = None
years = (11, 16, 17, 18)
for i, year in enumerate(years):
    df = extract_subset(data, year)
    if i == 0:
        changes = df
    elif i == 1:
        changes = changes[~changes.lad11cd.str.startswith('9')]
        df = df[~df.lad16cd.str.startswith('N')]
        changes = changes.merge(
            df,
            left_on='lad{}nm'.format(years[i-1]), 
            right_on='lad{}nm'.format(year), 
            how='outer'
        )
        changes = pandas.concat([changes, nimapping], axis=0)
    else:        
        changes = changes.merge(
            df,
            left_on='lad{}nm'.format(years[i-1]), 
            right_on='lad{}nm'.format(year), 
            how='outer'
        )
changes

In [ ]:
table = None
years = (11, 16, 17, 18)
for i, year in enumerate(years):
    df = data['lad20{}'.format(year)][['lad{}nm'.format(year), 'lad{}cd'.format(year)]]
    if i == 0:
        table = df
    elif i == 1:
        table = table[~table.lad11cd.str.startswith('9')]
        df = df[~df.lad16cd.str.startswith('N')]
        table = table.merge(
            df,
            left_on='lad{}nm'.format(years[i-1]), 
            right_on='lad{}nm'.format(year), 
            how='outer'
        )
        table = pandas.concat([table, nimapping], axis=0)
    else:        
        table = table.merge(
            df,
            left_on='lad{}nm'.format(years[i-1]), 
            right_on='lad{}nm'.format(year), 
            how='outer'
        )
table

In [ ]:
table.to_csv('lad_nmcd_changes.csv')

In [ ]:
lad11cd_to_lad16cd = {
    'E06000048': 'E06000057', # Northumberland
    'E07000100': 'E07000240', # St Albans
    'E07000104': 'E07000241', # Welwyn Hatfield
    'E07000097': 'E07000242', # East Hertfordshire
    'E07000101': 'E07000243', # Stevenage
    'E08000020': 'E08000037', # Gateshead
}
lad17cd_to_lad18cd = {
    'S12000015': 'S12000047', # Fife
    'S12000024': 'S12000048', # Perth and Kinross
}
lad17nm_to_lad18nm = {
    'Shepway': 'Folkestone and Hythe' # E07000112
}